In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY_GROQ"]=os.getenv("LANGCHAIN_API_KEY_GROQ")
os.environ["LANGCHAIN_TRACING"]="true"
os.environ["LANGCHAIN_PROJECT_GROQ"]=os.getenv("LANGCHAIN_PROJECT_GROQ")

In [6]:
# calling model
from langchain_groq import ChatGroq
model=ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

In [5]:
# define the prompt
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    [
        ("system" , "You are a teacher and you have to create {noofquestion} multiple choice question which have 4 option based on provide context {context} and set difficulties level according to the user {medium}")
    ]
)
